In [ ]:
# Load the two CSV files into dataframes
df1 = pd.read_csv("gt.csv")  # 1
df2 = pd.read_csv("annotations_archive.csv")  # 2

# Merge on filename and bbox columns
merged_df = df1.merge(df2, on=["filename", "bbox"], suffixes=("_file1", "_file2"))

# Check for mismatches with the added condition for species_id
mismatch = merged_df[
    (
        (merged_df["viewpoint_file1"] != merged_df["viewpoint_file2"])
        & ~(
            (merged_df["species_id_file1"] == "other")
            & (merged_df["species_id_file2"] == "other")
        ) & ~(
            (merged_df["ia_annotation_file1"] == False)
            & (merged_df["ia_annotation_file2"] == False)
        ) & ~(
            (merged_df["species_id_file1"] == "Garbage")
            & (merged_df["species_id_file2"] == "Garbage")
        ) 
    )
    | (merged_df["ia_annotation_file1"] != merged_df["ia_annotation_file2"])
    | (merged_df["species_id_file1"] != merged_df["species_id_file2"])
]

# Print mismatched entries
if not mismatch.empty:
    for _, row in mismatch.iterrows():
        if row["viewpoint_file1"] != row["viewpoint_file2"]:
            print(
                f"  Viewpoint: {row['viewpoint_file1']} (file1) vs {row['viewpoint_file2']} (file2)"
            )
        if row["ia_annotation_file1"] != row["ia_annotation_file2"]:
            print(
                f"  IA Annotation: {row['ia_annotation_file1']} (file1) vs {row['ia_annotation_file2']} (file2)"
            )
        if row["species_id_file1"] != row["species_id_file2"]:
            print(
                f"  Species ID: {row['species_id_file1']} (file1) vs {row['species_id_file2']} (file2)"
            )
        print("-" * 50)
        # Display the corresponding image
        image_path = f"test_dataset/images/{row['filename']}"  # Adjust this path as needed
        image = cv2.imread(image_path)
        
        if image is not None:
            # Extract bbox coordinates and crop the image
            bbox_cleaned = re.sub(r'[^\d.,]', '', row['bbox'])
            xmin, ymin, xmax, ymax = map(float, bbox_cleaned.split(','))
            xmin, ymin, xmax, ymax = int(xmin), int(ymin), int(xmax), int(ymax)
            cropped_image = image[ymin:ymax, xmin:xmax]
            
            # Display the cropped image
            plt.imshow(cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB))  # Convert BGR to RGB
            plt.title(f"Filename: {row['filename']}, Cropped BBox: {row['bbox']}")
            plt.axis("off")  # Hide axes for a cleaner look
            plt.show()
        else:
            print(f"Image {image_path} not found.")
else:
    print("No mismatches found.")
